# Exploring The Pizza Shop Market in Austin, Texas


This project aims to analyzes the local business needs of the great city of Austin, Texas, whos local slogan "Keep Austin Weird" was inspired by the cities small business alliance Austin Indepedent Business Alliance to promote small businesses!

While Austin is a booming metropolis City, we aim to analyze whether a pizza shop or a bike shop would be more lucrative!
Here you will find the analysis for Pizza Parlors!
![Pizza vs. Bike Shops in Austin](https://upload.wikimedia.org/wikipedia/commons/a/a5/Austin_Evening.jpg)<img src="url" width="100" height="100">


In [1]:
import json
import requests
import sys
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


# from helpers import *

ModuleNotFoundError: No module named 'helpers'

In [ ]:
url= 'https://api.yelp.com/v3/businesses/search'

In [ ]:
client_id = 'UPd8KVfQybexrmKSjNF-mA'
api_key = 'TQ3V7mYVhRo1vOV7HFkQEuG-QKC7eVgNjpOkfkGD43EnLnLLY2ub-owG779hx-vZI6YX6YQZSxmE9kOjoo8iQYMx3mq0pwqenQ3Jc_vYjMZCci2PPC-BevZVmZomYHYx'




___

## Step 1 -  compares the the number of pizza restaurants by zipcodes and ratings:



**Objective: We want to determine if there is an opporunity for expansion within local neighborhoods!

In [ ]:
headers = {'Authorization':'Bearer {}'.format(api_key),}

In [ ]:
term= 'restaurants'
location='Austin'
categories='pizza'

In [ ]:
url_params = {
                "term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "categories" : categories,
                "limit": 50,
    
            }

In [ ]:
def yelp_call(url_params, api_key):
    
    response= requests.get(url, headers=headers, params=url_params)  
    
    if response.status_code==200:
        return response.json()
     
    else: 
        return response.status_code
    


In [ ]:
response= yelp_call(url_params, api_key)

## Step 2: Determine how many pizza shops in Austin are chain Restaurants?

**Objective: We would like to see in which neighborhoods there is a need for a solid local pizza parlor.
Small businesses are the heart of Austin afterall!


In [ ]:
# setting business data equal to 'business' key in response dictionary
business_data= response['businesses']

**What data do we want from each business?**

In [ ]:
def parse_results(results):
    parsed_results =[]
    for biz in results:
        biz_info= ( biz['id'],
                   biz['name'],
                   biz['rating'],
                   biz['review_count'],
                   biz['location']['zip_code'])
        parsed_results.append(biz_info)
    
    return parsed_results



In [ ]:
parsed_results= parse_results(business_data)


In [ ]:
def df_save(csv_filepath, parsed_results):
    
    
    pr_df= pd.DataFrame(parsed_results, columns= 
             ['business_id','biz_name', 'biz_rating', 'biz_review_count', 'biz_zipcode'])
    
    pr_csv=pr_df.to_csv(path_or_buf=csv_filepath, mode="a", header="False")
   
    return print("Results added!")
   

In [ ]:
def yelp_data(url_params, api_key):
    yelp_call(url_params, api_key)
    parse_results(business_data)
    df_save('/Users/leratsayukova/Documents/Flatiron/phase_1/Bikes_Repo/Data_Exploration_Project/data/csv_data_pizza', parsed_results)
    our_data= pd.read_csv('/Users/leratsayukova/Documents/Flatiron/phase_1/Bikes_Repo/Data_Exploration_Project/data/csv_data_pizza')
    return our_data

In [ ]:
yelp_data(url_params,api_key)

In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0
num= response['total']

#set up a while loop to go through and grab the result 
while cur <= num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results =  yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    new_parsed_results = parse_results(results['businesses'])
    
    # use your function to insert your parsed results into the db
    df_save('/Users/leratsayukova/Documents/Flatiron/Bikes_Repo/data/csv_data_pizza', new_parsed_results)
    
    
    #increment the counter by 50 to move on to the next results
    cur += 50

In [ ]:
bikes_csv= pd.read_csv('/Users/leratsayukova/Documents/Flatiron/Bikes_Repo/data/csv_data_pizza', header=0, 
                       names=['business_id','biz_name', 'biz_rating', 'biz_review_count', 'biz_zipcode']).reset_index(drop=True)
# pd.set_option('display.max_rows', None, 'display.max_columns', None)
# drops rows
Bikes_csv= bikes_csv.drop([50,101,152])



In [ ]:
biz_ids=Bikes_csv.loc[:,'business_id'].to_list()
biz_ids_lst=list(set(biz_ids))


___

In [ ]:
def yelp_call_reviews(url_review, api_key):
    response= requests.get(url_reviews, headers=headers)  
    if response.status_code==200:
        return response.json()
    else: 
        return response.status_code


In [ ]:
def yelp_reviews(url_params_reviews, api_key):
    yelp_call_reviews(biz_ids_lst, api_key)
    parse_reviews(reviews)
    df_save_reviews('/Users/leratsayukova/Documents/Flatiron/Bikes_Repo/data/csv_reviews_pizza', parsed_reviews)
    our_data= pd.read_csv('/Users/leratsayukova/Documents/Flatiron/Bikes_Repo/data/csv_reviews_pizza')
    return our_data

In [ ]:
def parse_reviews(bid, reviews):
    parsed_reviews =[]
    for review in reviews:
        review_info= ( bid,
                    review['id'],
                    review['text'],
                    review['rating'] 
                           )
        parsed_reviews.append(review_info)
        
    return parsed_reviews


In [ ]:
def df_save_reviews(csv_filepath, parsed_reviews):
    
    
    pr_df= pd.DataFrame(parsed_reviews, columns= 
             ['business_id','id','text', 'rating'])
    
    pr_csv=pr_df.to_csv(path_or_buf=csv_filepath, mode="a", header=False)
   
    return print("Results added!")
            

In [ ]:
for bid in biz_ids_lst:
# #     print(bid)
    url_reviews='https://api.yelp.com/v3/businesses/' + bid + '/reviews'
    url_data = yelp_call_reviews(url_reviews, api_key)
    new_reviews = yelp_call_reviews(bid, api_key)
    #after you get your results you can now use your function to parse those results
    new_parsed_reviews = parse_reviews(bid, new_reviews['reviews'])
     # use your function to insert your parsed results into the db
    df_save_reviews('/Users/leratsayukova/Documents/Flatiron/Bikes_Repo/data/csv_reviews_pizza', new_parsed_reviews)
   

In [ ]:
Reviews_csv=pd.read_csv('/Users/leratsayukova/Documents/Flatiron/Bikes_Repo/data/csv_reviews_pizza', 
            names=['business_id','review_id', 'text', 'rating']).reset_index(drop=True)

## Part 4 -  Using python and pandas, write code to answer the questions below. 


- Which are the 5 most reviewed businesses in your dataset?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What percentage of your businesseshave a price label of one dollar sign? Two dollar signs? Three dollar signs? No dollar signs?
- Return the text of the reviews for the most reviewed business. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the business with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the business with the least reviews. 


In [ ]:
# Bikes_csv

In [ ]:
# merges data csv and reviews csv on 'business_id' column
merged_inner = pd.merge(left=Bikes_csv, right=Reviews_csv, left_on='business_id', right_on='business_id')


In [ ]:
mi=merged_inner.copy()

________________________

## PANDAS QUESTIONS

**What are the 5 most reviewed businesses

In [ ]:
# Renamed Merged DataFrame to 'mi' and checking for dtypes of columns
mi.dtypes
# Converts all the appropriate columns in our dataFrame to numeric types
# so that we can work with values
mi[['biz_rating', 'biz_review_count', 'biz_zipcode', 'rating']]=mi[[
    'biz_rating', 'biz_review_count', 'biz_zipcode', 'rating']].apply(pd.to_numeric)
mi.dtypes

In [ ]:
# Returns the 5 most reviewed restaurants and sets it equal to new variable
fivemr= mi.nlargest(15,['biz_review_count'])
fivemr.head(15)

**5 Most Reviewed Businesses**


In [ ]:
# Returns unique names of 5 most reviewed restaurants
fivemr_unique= fivemr['biz_name'].unique().tolist()
print("The 5 most reviewed restauraunts are: \n")
fivemr_unique

______

**What is the highest rating recieved in your data set and how many businesses have that rating?**


In [ ]:
# Singles out ratings column to see how many business have 5 stars
# Same data, nicer look. 
pd.DataFrame(mi.loc[:,'biz_rating'].value_counts())

___________

**What percentage of businesses have a rating greater than or 4.5? \
What percentage of businesses have a rating less than 3?**

In [ ]:
ratings= mi['biz_rating'].value_counts(normalize=True)*100
rating_prct= pd.DataFrame(ratings)
rating_prct

**18% = 4.5 stars or above** -approximately 18% of pizza shops have ratings with a 4.5 or above

In [ ]:
len(biz_ids_lst)

___________

**Find the highest rated business and return text of the most recent review. 
If multiple business have the same rating, select the business with the most reviews

**Highest Rated Business and Review**

In [ ]:
# mi.groupby(['biz_rating']).agg({'biz_review_count': max})
rat_five= mi[mi['biz_rating']==5]
rat_five_top=rat_five.nlargest(1,['biz_review_count'])
rat_five_top

In [ ]:
rat_five_top.loc[:,'text']

**Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the business with the least reviews. 


In [ ]:
rat_five_2= mi[mi['biz_rating']==1]

In [ ]:
rat_bot=rat_five_2.nsmallest(3,'biz_review_count')
rat_bot

**Lowest Rated Business and Review**

In [ ]:
rbarray=rat_bot['biz_name'].values
bottom3_lst= rbarray.tolist()
bottom3_lst

___________________

## Visualizations

## Step 5 -  Determine which neighborhoods need more pizza. 

Objective: Feed the people.

Finding the number of pizza parlors by zipcode

In [ ]:
# Get total number of zipcodes
zip_lst=(mi['biz_zipcode'].unique()).tolist()
len(zip_lst)
# Convert zipcodes in our list to strings
zip_lst_string=list(map(str, zip_lst))

In [ ]:
searchfor = ['Domino', 'Cicis', 'Chuck', 'Papa', 'MOD', 'Caesars', 'California', 'Hut']
no_chains= mi[~mi.biz_name.str.contains('|'.join(searchfor))]
mi['biz_name'].describe()

In [ ]:
no_chains['biz_name'].describe()

In [ ]:
#TOTAL NUMBER OF NON-CHAIN RESTAURANTS PER ZIPCODE
Without_Chains = no_chains['biz_zipcode'].value_counts()
#TOTAL NUMBER OF RESTAURANTS PER ZIPCODE
With_Chains = mi['biz_zipcode'].value_counts()
#INSERTING BOTH OBJECTS INTO A NEW DATAFRAME
df = pd.DataFrame({'Without Chains': Without_Chains,
                   'With Chains': With_Chains})
#PLOTTING THE DATAFRAME
ax = df.plot.bar(figsize=(10,5))

ax.set_title('Comparing Number of Non-Chain Restaurants Per Zip Code ')
ax.set_xlabel('Zipcode')
ax.set_ylabel('Number of Pizza Shops')
plt.tight_layout()
plt.savefig('Comp_Non_Pizza_Chains.png');

In [ ]:
# Sorts the number of pizzashops by zipcode and avgerage rating
biz_zipcodes= pd.DataFrame(mi.groupby('biz_zipcode')['biz_rating'].mean())
#Add New Column Equal to The Number Of Pizza Shops 
biz_zipcodes['#_pizzashops']= mi['biz_zipcode'].value_counts()
biz_zipcodes.sort_values(by=['#_pizzashops'])

In [ ]:
#Calling biz_zipcodes to check if cell above ran correctly
biz_zipcodes

In [ ]:
#Plotting Avg Biz Rating by Zip Code
axes = biz_zipcodes['biz_rating'].plot.bar(rot=90, x='biz_rating')
#Setting Chart Title
axes.set_title('Average Pizza Shop Rating by Zipcode')
#Setting X-axis label
axes.set_xlabel('Zipcode')
#Setting Y-axis label
axes.set_ylabel('Avg. Rating')

plt.tight_layout()
plt.savefig('Avg_Rtng_by_Zip.png');

## Conclusions

- **The market for bike shops in Austin is already saturated.
- **There are at least 5 nieighborhood markets which could benefit from a local pizza shop!
- **Based on GeoData there is at least one zipcode in Austin that could net great potential profits!